### Setup libraries

In [1]:
# !pip install deeppavlov

### Import libraries

In [8]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:72.5% !important; }</style>"))

In [9]:
import tensorflow as tf

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import os
import io
import re
import time
import unicodedata

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [10]:
plt.rcParams.update({'font.size': 14})
pd.set_option('precision', 3)
pd.set_option('max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('max_colwidth', 300)

In [11]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#     raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

### Paths to directories and files

In [12]:
DRAFT_DATASET_PATH = 'D:/kaggle/mailru/Otvety.txt'

### Loading data

In [13]:
lines = io.open(DRAFT_DATASET_PATH, encoding='UTF-8').read().strip().split('\n')

MemoryError: 

### Preprocessing data

In [ ]:
TAG_RE = re.compile(r'<[^>]+>')
# TAG_RE = re.compile(r'<.*?>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [ ]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    sentence = sentence.lower().strip()

    # Remove punctuations and numbers
    sentence = re.sub('[^А-Яа-я0-9-ё,.!?]', ' ', sentence)

    # Single character removal
#     sentence = re.sub(r'\s+[А-Яа-я0-9]\s+', ' ', sentence)

    # Removing multiple spaces
#     sentence = re.sub(r'\s+', ' ', sentence)

    sentence = re.sub(':\)', '', sentence)
    sentence = re.sub('[)"]', '', sentence)  
    sentence = re.sub('\s*\?\s*\.', '?', sentence)
    sentence = re.sub('\s*\!\s*\.', '!', sentence)
    sentence = re.sub('\s*\.', '.', sentence)
    sentence = re.sub('\.+', '.', sentence)
    sentence = re.sub('\s*\.\s*\.', ' ', sentence)
    sentence = re.sub('\  +', ' ', sentence)
   
    return sentence

In [ ]:
text = preprocess_text(lines)

In [ ]:
def split_by_sentence(sent):
    sent = text.split('\n---\n')[1:]
    
    inp = []
    tar = []

    for se in sent:
        se = se.split('\n')
        inp.append(se[0].strip())
        tar.append(' '.join([f'<start> {s}<stop>' for s in se[1:]]))
    
    return inp, tar

In [ ]:
inp, tar = split_by_sentence(text)
inp[0], tar[0]

In [11]:
# def preprocess_sentence(w):
#     w = w.lower().strip()

#     # creating a space between a word and the punctuation following it
#     # eg: "he is a boy." => "he is a boy ."
#     # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
#     w = re.sub(r"([?.!,¿])", r" \1 ", w)
#     w = re.sub(r'[" "]+', " ", w)

#     # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#     w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

#     w = w.strip()

#     # adding a start and an end token to the sentence
#     # so that the model know when to start and stop predicting.
#     w = '<start> ' + w + ' <end>'
#     return w

In [12]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [24]:


word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:]]

# return zip(*word_pairs)

In [25]:
lines

['',
 '\nвопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО ВАМ здесь кого советовали завести?)) . \nхомячка.... \nмужика, йопаря, собачку и 50 кошек))). \nОбщение !). \nпаучка. \nДа пол мне бы памыть!<br>А таг то ни чо. Типа ни каво!. \nя тут вообще что бы пообщаться..... \nА мне советовали сиси завести...))). \nНу, слава богу, мужика завести ещё не советовали))) А вот сватать к кому только не сватали). \nмне тут советовали завести любовника, мужа и много кошек )))<br>приветик ))). \n',
 '\nКак парни относятся к цветным линзам? Если у девушки то зеленые глаза, то голубые...)) . \nменя вобще прикалывает эта тема :). \nкогда этобыло редкость - было забавно, а когда все знают, что эта фальшивка, то уже не прикольно, как силиконовые сиськи или как налепленные синтетические волосы. \n',
 '\nЧто делать, сегодня нашёл 2 миллиона рублей? . \nЕсли это "счастье " действительно на вас свалилось, лучше пойти в милицию и заявить о находке. Такие деньги просто так не терют, а что самое интересное их не

In [26]:
word_pairs

[['<start>  <end>'],
 ['<start> вопрос о тдв давно и хорошо отдыхаем лично вам здесь кого советовали завести ? . хомячка . . . . мужика , йопаря , собачку и кошек . общение ! . паучка . да пол мне бы памыть ! br а таг то ни чо . типа ни каво ! . я тут вообще что бы пообщаться . . . . . а мне советовали сиси завести . . . . ну , слава богу , мужика завести ещ не советовали а вот сватать к кому только не сватали . мне тут советовали завести любовника , мужа и много кошек br приветик . <end>'],
 ['<start> как парни относятся к цветным линзам ? если у девушки то зеленые глаза , то голубые . . . . меня вобще прикалывает эта тема . когда этобыло редкость было забавно , а когда все знают , что эта фальшивка , то уже не прикольно , как силиконовые сиськи или как налепленные синтетические волосы . <end>'],
 ['<start> что делать , сегодня наш л миллиона рублей ? . если это счастье действительно на вас свалилось , лучше пойти в милицию и заявить о находке . такие деньги просто так не терют , а чт

In [14]:
en, ru = create_dataset(DRAFT_DATASET_PATH, None)
print(en[-1])
print(ru[-1])

ValueError: not enough values to unpack (expected 2, got 1)